In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [2]:
"""
   Initialize the NDI Aurora EM tracker
"""
SETTINGS_EM = {
    "tracker type": "aurora",
    "port": "COM3"
}
tracker_AURORA = NDITracker(SETTINGS_EM)
tracker_AURORA.start_tracking()

In [ ]:
"""
    Initialize the NDI Polaris Optical tracker (Track Butterfly Plate)
    MAKE SURE you have the Butterfly .rom file and its path
"""
SETTINGS_BF = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,

    # Windows Path
    "romfiles": ["PATH TO BUTTERFLY PLATE .ROM FILE"]
}
tracker_POLARIS = NDITracker(SETTINGS_BF)
tracker_POLARIS.start_tracking()

In [ ]:
BF_to_EN = np.eye(4)  # Fixed relationship, between Butterfly and Endoscope Coord. Systems, Using Yuan's Old Settings
EM_to_PI = np.eye(4)  # Fixed relationship, between EM Coil and Pin Coord. System.

PO_to_CT_calibration = np.load("PO_to_CT_calibration.npy")

CT_to_PO_calibration = np.linalg.inv(PO_to_CT_calibration)

FG_to_CT_calibration = np.load("FG_to_CT_calibration.npy")

CT_to_FG_calibration = np.linalg.inv(FG_to_CT_calibration)

PO_to_FG = np.dot(PO_to_CT_calibration, CT_to_FG_calibration)

In [10]:
while True:
    # Read new data from Aurora
    EM_handles, EM_timestamps, EM_framenumbers, EM_tracking, EM_quality = tracker_AURORA.get_frame()

    # Read new data from Polaris
    PO_handles, PO_timestamps, PO_framenumbers, PO_tracking, PO_quality = tracker_POLARIS.get_frame()

    # Update EM data
    FG_to_EM_updated = EM_tracking[0]

    # Update Endoscope Butterfly in Polaris
    PO_to_BF = PO_tracking[0]

    # Update Endoscope Coord. System in Polaris
    PO_to_EN = np.dot(PO_to_BF, BF_to_EN)

    # Update Endoscope in Phantom/CT Coord. System
    CT_to_EN = np.dot(CT_to_PO_calibration, PO_to_EN)

    # Update PIN in Phantom/CT Coord. System
    FG_to_PI = np.dot(FG_to_EM_updated, EM_to_PI)
    CT_to_PI = np.dot(CT_to_FG_calibration, FG_to_PI)